In [1]:
! pip install requests pandas gspread

In [2]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from tapdb import TapDBClient
from gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS

target_date = (date.today() - timedelta(days=1)).isoformat()
target_date = '2024-06-28'

tapdb = TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

target_date = tapdb.target_date.isoformat()
sheet = DateSheet('[RP_EN] Revenue Report', date=target_date)

In [3]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入', 'income', int),
    ('付費人數','paidPlayers', int),
    ('付費次數','payTimes', int),
    ('ARPU','ARPU', float),
    ('ARPPU','ARPPU', float),
    ('付費率','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,16075,579,1422,0,0,0,0.97,27.76,3.49


In [4]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,17634,3199,14435,13622,12044,10326,7866


In [5]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入n2i', 'income', int),
    ('付費人數n2i','paidPlayers', int),
    ('付費次數n2i','payTimes', int),
    ('ARPUn2i','ARPU', float),
    ('ARPPUn2i','ARPPU', float),
    ('付費率n2i','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,3005,90,228,0,0,0,1.57,33.39,4.71


In [6]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置n2i', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置n2i', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,4531,4525,6,0,0,0,0


In [11]:
#[RP_EN] Revenue Report >> 設備留存
data = tapdb.get_data('RP NTK (PRO)', 'retain', begin_date='2024-03-04', pf='and', type='both',
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv('~/Desktop/retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-03-03,80,0.4625,0.3750,0.3375,0.3250,0.3250,0.2625,0.3125,0.2500,0.2500,...,0.1500,0.1500,0.1625,0.1500,0.1125,0.0625,0.0500,0.0125,NaN,NaN
2024-03-04,7333,0.6464,0.5365,0.4987,0.4716,0.4519,0.4444,0.4259,0.3932,0.3803,...,0.2256,0.2187,0.2249,0.2112,0.2040,0.0997,0.0720,NaN,NaN,NaN
2024-03-05,9156,0.4811,0.4040,0.3727,0.3461,0.3336,0.3177,0.2994,0.2866,0.2704,...,0.1547,0.1556,0.1513,0.1430,0.1418,0.0667,0.0470,NaN,NaN,NaN
2024-03-06,7965,0.4359,0.3406,0.3101,0.2884,0.2719,0.2573,0.2467,0.2388,0.2213,...,0.1248,0.1163,0.1165,0.1105,0.1105,0.0551,0.0345,NaN,NaN,NaN
2024-03-07,7855,0.3990,0.3120,0.2768,0.2545,0.2354,0.2358,0.2199,0.2042,0.1947,...,0.1094,0.1034,0.1018,0.0965,0.0923,0.0447,0.0281,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-24,3421,0.2426,0.1485,0.1204,0.1064,0.0965,0.0865,0.0532,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-25,3481,0.2514,0.1474,0.1117,0.0962,0.0879,0.0577,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-26,3734,0.2378,0.1425,0.1178,0.0951,0.0597,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
